# Congressional Votes
- Pull Voting data by House Rep
- Join with Bill data on Bill ID
- Understand relationship between Bill sponsorship with party lines, ...

In [1]:
from urllib.request import urlopen
import bs4
import requests
from lxml import html
from lxml.cssselect import CSSSelector
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

## House Bill Data
- Identify bill_action range of values
- 

In [2]:
xml_range = range(1,5)
bill_rollnum = []
bill_chamber = []
bill_action = []
bill_date = []
bill_voteurl = []

In [3]:
for i in xml_range:
    bill_url = 'https://www.govinfo.gov/bulkdata/BILLSTATUS/115/hr/BILLSTATUS-115hr' + str(i) + '.xml'
    bill_source = requests.get(bill_url)
    bill_ntree = html.document_fromstring(bill_source.content)
    
    bill_rollnum_text = bill_ntree.xpath('//recordedvote[descendant::chamber/text()="House"]/rollnumber/text()')
    bill_rollnum.append(bill_rollnum_text)
    bill_chamber_text = bill_ntree.xpath('//recordedvote/chamber[text()="House"]/text()')
    bill_chamber.append(bill_chamber_text)
    bill_action_text = bill_ntree.xpath('//recordedvote[descendant::chamber/text()="House"]/fullactionname/text()')
    bill_action.append(bill_action_text)
    bill_date_text = bill_ntree.xpath('//recordedvote[descendant::chamber/text()="House"]/date/text()')
    bill_date.append(bill_date_text)
    bill_voteurl_text = bill_ntree.xpath('//recordedvote[descendant::chamber/text()="House"]/url/text()')
    bill_voteurl.append(bill_voteurl_text)

In [4]:
bill_rollnumf = [val for sublist in bill_rollnum for val in sublist]
bill_chamberf = [val for sublist in bill_chamber for val in sublist]
bill_actionf = [val for sublist in bill_action for val in sublist]
bill_datef = [val for sublist in bill_date for val in sublist]
bill_voteurlf = [val for sublist in bill_voteurl for val in sublist]

In [5]:
bill_voteurlf

['http://clerk.house.gov/evs/2017/roll699.xml',
 'http://clerk.house.gov/evs/2017/roll692.xml',
 'http://clerk.house.gov/evs/2017/roll691.xml',
 'http://clerk.house.gov/evs/2017/roll654.xml',
 'http://clerk.house.gov/evs/2017/roll653.xml',
 'http://clerk.house.gov/evs/2017/roll637.xml',
 'http://clerk.house.gov/evs/2018/roll205.xml',
 'http://clerk.house.gov/evs/2018/roll204.xml',
 'http://clerk.house.gov/evs/2018/roll243.xml',
 'http://clerk.house.gov/evs/2018/roll165.xml',
 'http://clerk.house.gov/evs/2018/roll164.xml']

In [6]:
votingdf = pd.DataFrame({'bill_rollnum':bill_rollnumf, 'bill_chamber':bill_chamberf, 'bill_action':bill_actionf, 
                         'bill_date':bill_datef, 'bill_voteurl':bill_voteurlf})

In [7]:
col_seq = ['bill_date','bill_rollnum','bill_chamber','bill_action','bill_voteurl']
votingdf = votingdf.reindex(columns=col_seq)

In [8]:
votingdf

,bill_date,bill_rollnum,bill_chamber,bill_action,bill_voteurl
0,2017-12-20T17:56:09Z,699,House,Vote on House Agreeing to the Senate Action,http://clerk.house.gov/evs/2017/roll699.xml
1,2017-12-19T19:28:52Z,692,House,Placeholder Text for H42510,http://clerk.house.gov/evs/2017/roll692.xml
2,2017-12-19T19:20:59Z,691,House,Placeholder Text for H42411,http://clerk.house.gov/evs/2017/roll691.xml
3,2017-12-05T01:55:25Z,654,House,Vote on House Agreeing to the Senate Action,http://clerk.house.gov/evs/2017/roll654.xml
4,2017-12-05T00:05:06Z,653,House,Vote on House Agreeing to the Senate Action,http://clerk.house.gov/evs/2017/roll653.xml
5,2017-11-16T18:49:45Z,637,House,Passage of a Measure,http://clerk.house.gov/evs/2017/roll637.xml
6,2018-05-18T16:04:34Z,205,House,Passage of a Measure,http://clerk.house.gov/evs/2018/roll205.xml
7,2018-05-18T15:57:08Z,204,House,Motion to Commit/Recommit With Instructions Re...,http://clerk.house.gov/evs/2018/roll204.xml
8,2018-06-08T01:18:53Z,243,House,Passage of a Measure,http://clerk.house.gov/evs/2018/roll243.xml
9,2018-04-27T15:39:30Z,165,House,Passage of a Measure,http://clerk.house.gov/evs/2018/roll165.xml


# House Vote Data - By Rep

In [107]:
voteurl_list = votingdf.bill_voteurl.tolist()
vote_legisnum = []
vote_nameid = []
vote_rollnum = []
vote_record = []

In [10]:
voteurl_list

['http://clerk.house.gov/evs/2017/roll699.xml',
 'http://clerk.house.gov/evs/2017/roll692.xml',
 'http://clerk.house.gov/evs/2017/roll691.xml',
 'http://clerk.house.gov/evs/2017/roll654.xml',
 'http://clerk.house.gov/evs/2017/roll653.xml',
 'http://clerk.house.gov/evs/2017/roll637.xml',
 'http://clerk.house.gov/evs/2018/roll205.xml',
 'http://clerk.house.gov/evs/2018/roll204.xml',
 'http://clerk.house.gov/evs/2018/roll243.xml',
 'http://clerk.house.gov/evs/2018/roll165.xml',
 'http://clerk.house.gov/evs/2018/roll164.xml']

In [108]:
for i in range(1,len(voteurl_list)):
    vote_url = '{}'.format(voteurl_list[i])
    vote_source = requests.get(vote_url)
    vote_ntree = html.document_fromstring(vote_source.content)
    
    vote_legisnum_text = vote_ntree.xpath('//legis-num/text()')
    vote_legisnum.append(vote_legisnum_text)
    vote_rollnum_text = vote_ntree.xpath('//rollcall-num/text()')
    vote_rollnum.append(vote_rollnum_text)
    vote_nameid_text = vote_ntree.xpath('//recorded-vote/legislator/@name-id')
    vote_nameid.append(vote_nameid_text)
    vote_record_text = vote_ntree.xpath('//recorded-vote/vote/text()')
    vote_record.append(vote_record_text)

In [12]:
votedf = pd.DataFrame({'vote_legisnum':vote_legisnum, 'vote_rollnum':vote_rollnum, 
                       'vote_nameid':vote_nameid, 'vote_record':vote_record})

In [13]:
col_seq = ['vote_legisnum','vote_rollnum','vote_nameid','vote_record']
votedf = votedf.reindex(columns=col_seq)

In [25]:
votedf

,vote_legisnum,vote_rollnum,vote_nameid,vote_record
0,[H R 1],[692],"[A000374, A000370, A000055, A000371, A000372, ...","[Yea, Nay, Yea, Nay, Yea, Yea, Yea, Yea, Yea, ..."
1,[H R 1],[691],"[A000374, A000370, A000055, A000371, A000372, ...","[Nay, Yea, Nay, Yea, Nay, Nay, Nay, Nay, Nay, ..."
2,[H R 1],[654],"[A000374, A000370, A000055, A000371, A000372, ...","[Nay, Yea, Nay, Yea, Nay, Nay, Nay, Nay, Nay, ..."
3,[H R 1],[653],"[A000374, A000370, A000055, A000371, A000372, ...","[Yea, Nay, Yea, Nay, Yea, Nay, Yea, Yea, Yea, ..."
4,[H R 1],[637],"[A000374, A000370, A000055, A000371, A000372, ...","[Yea, Nay, Yea, Nay, Yea, Yea, Yea, Yea, Yea, ..."
5,[H R 2],[205],"[A000374, A000370, A000055, A000371, A000372, ...","[Yea, Nay, Yea, Nay, Yea, Nay, Yea, Yea, Yea, ..."
6,[H R 2],[204],"[A000374, A000370, A000055, A000371, A000372, ...","[No, Aye, No, Aye, No, No, No, No, No, No, No,..."
7,[H R 3],[243],"[A000374, A000370, A000055, A000371, A000372, ...","[Yea, Nay, Yea, Nay, Yea, Yea, Yea, Yea, Yea, ..."
8,[H R 4],[165],"[A000374, A000370, A000055, A000371, A000372, ...","[Yea, Yea, Yea, Yea, Yea, Nay, Yea, Yea, Yea, ..."
9,[H R 4],[164],"[A000374, A000370, A000055, A000371, A000372, ...","[No, Aye, No, Aye, No, No, No, No, No, No, No,..."


In [129]:
dict1 = {}
dict2 = {}
for i in range(10):
    a = votedf.iloc[i].vote_rollnum
    b = votedf.iloc[i].vote_nameid
    c = votedf.iloc[i].vote_record

    d = list(zip(b, c))
    for x, y in d:
          dict2[x] = y
    
    dict1[str(a)] = dict2


In [131]:
dict1

{"['164']": {'A000055': 'No',
  'A000367': 'No',
  'A000369': 'No',
  'A000370': 'Aye',
  'A000371': 'Aye',
  'A000372': 'No',
  'A000374': 'No',
  'A000375': 'No',
  'B000213': 'No',
  'B000490': 'Aye',
  'B000574': 'Aye',
  'B000755': 'No',
  'B001227': 'Aye',
  'B001243': 'Not Voting',
  'B001248': 'No',
  'B001250': 'No',
  'B001251': 'Not Voting',
  'B001257': 'No',
  'B001260': 'No',
  'B001269': 'No',
  'B001270': 'Aye',
  'B001273': 'Not Voting',
  'B001274': 'No',
  'B001275': 'No',
  'B001278': 'Aye',
  'B001281': 'Aye',
  'B001282': 'No',
  'B001283': 'Yea',
  'B001284': 'No',
  'B001285': 'Aye',
  'B001286': 'Aye',
  'B001287': 'Aye',
  'B001289': 'No',
  'B001290': 'No',
  'B001291': 'No',
  'B001292': 'Aye',
  'B001293': 'No',
  'B001294': 'No',
  'B001295': 'No',
  'B001296': 'Aye',
  'B001297': 'No',
  'B001298': 'No',
  'B001299': 'No',
  'B001300': 'Aye',
  'B001301': 'No',
  'B001302': 'No',
  'B001303': 'Aye',
  'B001304': 'Aye',
  'B001305': 'No',
  'C000059': 'No'

In [119]:
dict1.keys()

dict_keys(["['692']", "['691']", "['654']", "['653']", "['637']", "['205']", "['204']", "['243']", "['165']", "['164']"])

### Data Schema
BILL OBJECT
- Legis Number
- Policy Area
- By Primary Sponsor
- By Party
- By State
- Date Proposed
- Date Passed

VOTE OBJECT
- Legis Number
- Roll Number
- Congressman
- Vote Record
- Date Roll Call Vote

CONGRESSMAN OBJECT
- Name
- Party
- State
- District
- Townname

### Data Query Requirements
- What is the voting record by Congressman? Filter by Congressman, Policy Area
- How bipartisan is each Congressman? Filter by votes given bill sponsorship
- What's the process for bill passage? % passed, # roll calls, length of time
- How 'active' / 'influential' is each Congressman?
- 


In [132]:
from sqlite3 import dbapi2 as sq3
import os
PATHSTART="."
def get_db(dbfile):
    sqlite_db = sq3.connect(os.path.join(PATHSTART, dbfile))
    return sqlite_db

In [134]:
def init_db(dbfile, schema):
    """Creates the database tables."""
    db = get_db(dbfile)
    db.cursor().executescript(schema)
    db.commit()
    return db

In [142]:
ourschema="""
DROP TABLE IF EXISTS "candidates";
DROP TABLE IF EXISTS "contributors";
CREATE TABLE "candidates" (
    "id" INTEGER PRIMARY KEY  NOT NULL ,
    "first_name" VARCHAR,
    "last_name" VARCHAR,
    "middle_name" VARCHAR,
    "party" VARCHAR NOT NULL
);
CREATE TABLE "contributors" (
    "id" INTEGER PRIMARY KEY  AUTOINCREMENT  NOT NULL,
    "last_name" VARCHAR,
    "first_name" VARCHAR,
    "middle_name" VARCHAR,
    "street_1" VARCHAR,
    "street_2" VARCHAR,
    "city" VARCHAR,
    "state" VARCHAR,
    "zip" VARCHAR,
    "amount" INTEGER,
    "date" DATETIME,
    "candidate_id" INTEGER NOT NULL,
    FOREIGN KEY(candidate_id) REFERENCES candidates(id)
);
"""

In [150]:
dfcand=pd.read_csv("./candidates.txt", sep='|')
dfcand.head()

,id,first_name,last_name,middle_name,party
0,33,Joseph,Biden,NaN,D
1,36,Samuel,Brownback,NaN,R
2,34,Hillary,Clinton,R.,D
3,39,Christopher,Dodd,J.,D
4,26,John,Edwards,NaN,D


In [151]:
dfcwci=pd.read_csv("./contributors_with_candidate_id.txt", sep="|")
dfcwci.head()

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,NaN,Agee,Steven,NaN,549 Laurel Branch Road,NaN,Floyd,VA,24091,500.0,2007-06-30,16
1,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,250.0,2007-05-16,16
2,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,50.0,2007-06-18,16
3,NaN,Ahrens,Don,NaN,4034 Rennellwood Way,NaN,Pleasanton,CA,94566,100.0,2007-06-21,16
4,NaN,Akin,Charles,NaN,10187 Sugar Creek Road,NaN,Bentonville,AR,72712,100.0,2007-06-16,16


In [156]:
db=init_db("cancont.db", ourschema)

In [157]:
dfcand.to_sql("candidates", db, if_exists="append", index=False)

In [158]:
dfcwci.to_sql("contributors", db, if_exists="append", index=False)

In [159]:
sel="""
SELECT * FROM candidates;
"""
c=db.cursor().execute(sel)

In [160]:
c.fetchall()

[(16, 'Mike', 'Huckabee', None, 'R'),
 (20, 'Barack', 'Obama', None, 'D'),
 (22, 'Rudolph', 'Giuliani', None, 'R'),
 (24, 'Mike', 'Gravel', None, 'D'),
 (26, 'John', 'Edwards', None, 'D'),
 (29, 'Bill', 'Richardson', None, 'D'),
 (30, 'Duncan', 'Hunter', None, 'R'),
 (31, 'Dennis', 'Kucinich', None, 'D'),
 (32, 'Ron', 'Paul', None, 'R'),
 (33, 'Joseph', 'Biden', None, 'D'),
 (34, 'Hillary', 'Clinton', 'R.', 'D'),
 (35, 'Mitt', 'Romney', None, 'R'),
 (36, 'Samuel', 'Brownback', None, 'R'),
 (37, 'John', 'McCain', None, 'R'),
 (38, 'Tom', 'Tancredo', None, 'R'),
 (39, 'Christopher', 'Dodd', 'J.', 'D'),
 (41, 'Fred', 'Thompson', 'D.', 'R')]

In [162]:
def make_query(sel):
    c=db.cursor().execute(sel)
    return c.fetchall()

In [167]:
cont_cols = [e[1] for e in make_query("PRAGMA table_info(contributors);")]
cont_cols

def make_frame(list_of_tuples, legend=cont_cols):
    framelist=[]
    for i, cname in enumerate(legend):
        framelist.append((cname,[e[i] for e in list_of_tuples]))
    return pd.DataFrame.from_items(framelist)

In [164]:
make_query("SELECT * FROM contributors;")

[(1,
  'Agee',
  'Steven',
  None,
  '549 Laurel Branch Road',
  None,
  'Floyd',
  'VA',
  '24091',
  500,
  '2007-06-30',
  16),
 (2,
  'Ahrens',
  'Don',
  None,
  '4034 Rennellwood Way',
  None,
  'Pleasanton',
  'CA',
  '94566',
  250,
  '2007-05-16',
  16),
 (3,
  'Ahrens',
  'Don',
  None,
  '4034 Rennellwood Way',
  None,
  'Pleasanton',
  'CA',
  '94566',
  50,
  '2007-06-18',
  16),
 (4,
  'Ahrens',
  'Don',
  None,
  '4034 Rennellwood Way',
  None,
  'Pleasanton',
  'CA',
  '94566',
  100,
  '2007-06-21',
  16),
 (5,
  'Akin',
  'Charles',
  None,
  '10187 Sugar Creek Road',
  None,
  'Bentonville',
  'AR',
  '72712',
  100,
  '2007-06-16',
  16),
 (6,
  'Akin',
  'Mike',
  None,
  '181 Baywood Lane',
  None,
  'Monticello',
  'AR',
  '71655',
  1500,
  '2007-05-18',
  16),
 (7,
  'Akin',
  'Rebecca',
  None,
  '181 Baywood Lane',
  None,
  'Monticello',
  'AR',
  '71655',
  500,
  '2007-05-18',
  16),
 (8,
  'Aldridge',
  'Brittni',
  None,
  '808 Capitol Square Place, SW',

In [169]:
out=make_query("SELECT * FROM contributors WHERE state='VA' AND amount < 400;")
make_frame(out)

,id,last_name,first_name,middle_name,street_1,street_2,city,state,zip,amount,date,candidate_id
0,28,Buckheit,Bruce,None,8904 KAREN DR,None,FAIRFAX,VA,220312731,100.00,2007-09-19,20
1,78,Ranganath,Anoop,None,2507 Willard Drive,None,Charlottesville,VA,22903,-100.00,2008-04-21,32
2,89,Perreault,Louise,None,503 Brockridge Hunt Drive,None,Hampton,VA,23666,-34.08,2008-04-21,32
3,146,ABDELLA,THOMAS,M.,4231 MONUMENT WALL WAY #340,None,FAIRFAX,VA,220308440,50.00,2007-09-30,35


## House Bios

In [13]:
https://xml.house.gov/MemberData/MemberData.xml

SyntaxError: invalid syntax (<ipython-input-13-0714b07055fd>, line 1)

In [181]:
party = ntree.xpath('//party/text()')
yea = ntree.xpath('//yea-total/text()')
nay = ntree.xpath('//nay-total/text()')
present = ntree.xpath('//present-total/text()')
not_voting = ntree.xpath('//not-voting-total/text()')

party = [*party, 'Total']

In [182]:
congress = ntree.xpath('//congress/text()')
rollnum = ntree.xpath('//rollcall-num/text()')

congress = [*congress, *congress, *congress, *congress]
rollnum = [*rollnum, *rollnum, *rollnum, *rollnum]

In [184]:
votingdf = pd.DataFrame({'congress':congress, 'rollnum':rollnum, 'party':party, 'yea':yea, 'nay':nay, 'present':present, 'not_voting':not_voting})
votingdf = votingdf[['congress', 'rollnum', 'party', 'yea', 'nay', 'present', 'not_voting']]

In [185]:
votingdf

,congress,rollnum,party,yea,nay,present,not_voting
0,115,699,Republican,224,12,0,3
1,115,699,Democratic,0,189,0,4
2,115,699,Independent,0,0,0,0
3,115,699,Total,224,201,0,7


In [174]:
votingdf_flat = pd.melt(votingdf, id_vars=['congress','rollnum','party'], 
                        value_vars=['yea','nay','present','not_voting'], 
                        var_name='vote_type', value_name='vote_count')

In [ ]:
votingdf_flat

In [186]:
votingdf_699 = votingdf

In [179]:
votingdf_692

,congress,rollnum,party,yea,nay,present,not_voting
0,115,692,Republican,227,12,0,0
1,115,692,Democratic,0,191,0,2
2,115,692,Independent,0,0,0,0
3,115,692,Total,227,203,0,2


In [187]:
votingdf_699

,congress,rollnum,party,yea,nay,present,not_voting
0,115,699,Republican,224,12,0,3
1,115,699,Democratic,0,189,0,4
2,115,699,Independent,0,0,0,0
3,115,699,Total,224,201,0,7


In [171]:
# binary can never represent 0.1, which is why it is numerically unstable
count = 0

In [172]:
for _ in range(1000):
    count += .1
count

99.9999999999986